In [ ]:
'''
get pricing data
get stochastic data
filter for 8am and 12pm rows
make open price the 8am price (shift open price down 1)
shift stochastics down 1
change time - 7 hours
separate datetime into date and hour



'''

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
from datetime import datetime, timedelta

In [2]:
def filter_(df, col, val):
    return df.loc[df[col] == val]

## filter on multiple columns for a single value per column - dictionary of {column:value} pairs
def filter_more_cols(df, dict: dict):
    cols = list(dict.keys())
    vals = list(dict.values())
    length = len(dict)

    x = filter_(df, cols[0], vals[0])
    if length == 1:
        return x
    if length > 1:
        for i in range(1, length):
            x = filter_(x, cols[i], vals[i])
        return x

## filter single column for multiple values - can retain or remove specified values
def filter_more_values(df, col, val: list, neg = False):
    length = len(val)

    if neg == False:
        val1 = df.loc[df[col] == val[0]]
        if length == 1:
            return val1
        else:
            input = []
            input.append(val1)
            for i in range(1, length):
                val2 = df.loc[df[col] == val[i]]
                input.append(val2)
            outputFalse = pd.concat(input)
            return outputFalse

    elif neg == True:
        val1 = df.loc[df[col] != val[0]]
        if length == 1:
            return val1
        else:
            for i in range(1, length):
                val1 = val1.loc[val1[col] != val[i]]
        return val1


def filter_all(df, d:dict):
    for i, v in enumerate(d):
        df = filter_more_values(df, list(d)[i], d.get(v))
    return df

In [64]:
output = pd.DataFrame()
o_ts = [10]
c_ts = [22]


open_time = 8
close_time = 20

for open_time in o_ts:

    for close_time in c_ts:

        df = yf.download('ES=F', start='2022-04-01', end='2023-01-01', interval='60m')

        # Define periods
        k_period = 14
        d_period = 5
        # Adds a "n_high" column with max value of previous 14 periods
        df['n_high'] = df['High'].rolling(k_period).max()
        # Adds an "n_low" column with min value of previous 14 periods
        df['n_low'] = df['Low'].rolling(k_period).min()
        # Uses the min/max values to calculate the %k (as a percentage)
        df['%K'] = (df['Close'] - df['n_low']) * 100 / (df['n_high'] - df['n_low'])
        # Uses the %k to calculates a SMA over the past 3 values of %k
        df['%D'] = df['%K'].rolling(d_period).mean()

        stochastic = df.ta.stoch(high='high', low='low', k=14, d=3, append=True)

        pricing = df[['Open', 'Close']]

        data = pd.concat([pricing, stochastic], axis=1)

        data = data.rename(columns={'STOCHk_14_3_3': 'k', 'STOCHd_14_3_3': 'd'})

        data = data.dropna()

        data = data.reset_index()
        data['Datetime'] = data['Datetime'] + timedelta(hours = -7)
        data['date'] = [x.date() for x in data['Datetime']]
        data['hour'] = [x.hour for x in data['Datetime']]

        data = data[['date', 'hour', 'Open', 'Close', 'k', 'd']]

        data = filter_all(data, {'hour': [open_time, close_time]})

        data = data.sort_values(by=['date', 'hour'])

        data['Open'] = data['Open'].shift(1)
        data['k'] = data['k'].shift(1)
        data['d'] = data['d'].shift(1)

        data = filter_all(data, {'hour': [close_time]})

        data['price_change'] = data['Close'] - data['Open']
        data['stochastic_change'] = data['k'] - data['d']
        data['price_direction'] = [1 if x > 0 else 0 for x in data['price_change']]
        data['stochastic_direction'] = [1 if x > 0 else 0 for x in data['stochastic_change']]

        data['agreement'] = [1 if x == y else 0 for x, y in zip(data['price_direction'], data['stochastic_direction'])]

        data['price_change'] = data['price_change'].abs()

        data['price_change'] = data['price_change'].abs()

        probability = (data.groupby(['agreement'])['agreement'].count()/data['agreement'].count())
        probability = pd.DataFrame(probability).set_index('agreement')
        probability['index'] = [0, 1]
        probability = probability.reset_index()
        probability[['index', 'agreement']]

        avg_trade = data.groupby(['agreement'])['price_change'].mean()
        avg_trade = avg_trade.reset_index()

        probability = probability.rename(columns={'agreement': 'probability'})

        matrix = pd.concat([avg_trade, probability], axis=1)
        matrix = matrix.iloc[:, [1, 2]]

        matrix['expected_value'] = matrix['price_change'] * matrix['probability']

        matrix = matrix.rename(columns={'price_change': 'Average Price Move'})
        matrix['Outcome'] = ['Loss', 'Win']

        matrix = matrix[['Outcome', 'Average Price Move', 'probability', 'expected_value']]

        print(f'Trade open at {open_time} and closing at {close_time}')
        matrix

        output_data = matrix.copy()
        output_data['open'], output_data['close'] = open_time, close_time

        output = pd.concat([output, output_data])

output


[*********************100%***********************]  1 of 1 completed
Trade open at 10 and closing at 22


,Outcome,Average Price Move,probability,expected_value,open,close
0,Loss,39.113372,0.450262,17.611257,10,22
1,Win,52.651442,0.549738,28.944510,10,22


In [60]:
output['diff_in_ev'] = output['expected_value'].diff()

In [63]:
filter_all(output, {'Outcome': ['Win']}).to_clipboard(index=False)

In [4]:
df = yf.download('ES=F', start='2022-04-01', end='2023-01-01', interval='60m')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-04-01 00:00:00,4546.25,4547.00,4542.00,4542.50,4542.50,0
2022-04-01 01:00:00,4542.75,4546.00,4539.50,4539.50,4539.50,5995
2022-04-01 02:00:00,4539.50,4548.75,4538.25,4547.25,4547.25,9384
2022-04-01 03:00:00,4547.25,4551.25,4540.75,4547.00,4547.00,16340
2022-04-01 04:00:00,4547.00,4550.75,4545.75,4549.75,4549.75,8823
...,...,...,...,...,...,...
2022-12-30 12:00:00,3845.25,3848.50,3841.25,3847.50,3847.50,70104
2022-12-30 13:00:00,3847.50,3847.75,3821.50,3830.25,3830.25,134666
2022-12-30 14:00:00,3830.50,3832.50,3823.00,3828.00,3828.00,101782


In [5]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [6]:
# Define periods
k_period = 14
d_period = 5
# Adds a "n_high" column with max value of previous 14 periods
df['n_high'] = df['High'].rolling(k_period).max()
# Adds an "n_low" column with min value of previous 14 periods
df['n_low'] = df['Low'].rolling(k_period).min()
# Uses the min/max values to calculate the %k (as a percentage)
df['%K'] = (df['Close'] - df['n_low']) * 100 / (df['n_high'] - df['n_low'])
# Uses the %k to calculates a SMA over the past 3 values of %k
df['%D'] = df['%K'].rolling(d_period).mean()

In [7]:
stochastic = df.ta.stoch(high='high', low='low', k=14, d=3, append=True)

In [8]:
pricing = df[['Open', 'Close']]

In [9]:
data = pd.concat([pricing, stochastic], axis=1)

In [10]:
data = data.rename(columns={'STOCHk_14_3_3': 'k', 'STOCHd_14_3_3': 'd'})

In [11]:
data = data.dropna()

In [12]:
data = data.reset_index()
data['Datetime'] = data['Datetime'] + timedelta(hours = -7)
data['date'] = [x.date() for x in data['Datetime']]
data['hour'] = [x.hour for x in data['Datetime']]

In [13]:
data = data[['date', 'hour', 'Open', 'Close', 'k', 'd']]

In [14]:
data = filter_all(data, {'hour': [8, close_time]})

In [15]:
data = data.sort_values(by=['date', 'hour'])

In [16]:
data['Open'] = data['Open'].shift(1)
data['k'] = data['k'].shift(1)
data['d'] = data['d'].shift(1)

In [17]:
data = filter_all(data, {'hour': [close_time]})

In [18]:
data['price_change'] = data['Close'] - data['Open']
data['stochastic_change'] = data['k'] - data['d']
data['price_direction'] = [1 if x > 0 else 0 for x in data['price_change']]
data['stochastic_direction'] = [1 if x > 0 else 0 for x in data['stochastic_change']]

In [19]:
data['agreement'] = [1 if x == y else 0 for x, y in zip(data['price_direction'], data['stochastic_direction'])]
data

,date,hour,Open,Close,k,d,price_change,stochastic_change,price_direction,stochastic_direction,agreement
2,2022-04-03,13,NaN,4532.00,NaN,NaN,NaN,NaN,0,0,1
25,2022-04-04,13,4570.50,4573.00,97.260657,92.569858,2.50,4.690799,1,1,1
48,2022-04-05,13,4534.50,4512.50,9.923251,9.722959,-22.00,0.200293,0,1,0
71,2022-04-06,13,4489.75,4462.25,37.337769,27.579343,-27.50,9.758426,0,1,0
94,2022-04-07,13,4501.75,4495.00,81.536621,58.728534,-6.75,22.808087,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
4267,2022-12-22,13,3826.00,3853.50,27.035623,14.816924,27.50,12.218700,1,1,1
4290,2022-12-26,13,3867.25,3898.75,86.254904,79.663741,31.50,6.591163,1,1,1
4313,2022-12-27,13,3854.50,3860.50,26.745718,30.830040,6.00,-4.084321,1,0,0
4336,2022-12-28,13,3824.75,3808.75,14.947215,19.585051,-16.00,-4.637836,0,0,1


In [20]:
data['price_change'] = data['price_change'].abs()

In [21]:
probability = (data.groupby(['agreement'])['agreement'].count()/data['agreement'].count())
probability = pd.DataFrame(probability).set_index('agreement')
probability['index'] = [0, 1]
probability = probability.reset_index()
probability[['index', 'agreement']]
probability

,agreement,index
0,0.369792,0
1,0.630208,1


In [22]:
avg_trade = data.groupby(['agreement'])['price_change'].mean()

In [23]:
avg_trade = avg_trade.reset_index()
avg_trade

,agreement,price_change
0,0,15.422535
1,1,21.227083


In [24]:
probability = probability.rename(columns={'agreement': 'probability'})

In [25]:
# probability
probability[['index', 'probability']]

,index,probability
0,0,0.369792
1,1,0.630208


In [26]:
matrix = pd.concat([avg_trade, probability], axis=1)
matrix = matrix.iloc[:, [1, 2]]

In [27]:
matrix['expected_value'] = matrix['price_change'] * matrix['probability']
matrix

,price_change,probability,expected_value
0,15.422535,0.369792,5.703125
1,21.227083,0.630208,13.377485


In [225]:
matrix = matrix.rename(columns={'price_change': 'Average Price Move'})
matrix['Outcome'] = ['Loss', 'Win']
matrix

,Average Price Move,probability,expected_value,Outcome
0,13.500000,0.375,5.062500,Loss
1,20.957983,0.625,13.098739,Win


In [226]:
matrix = matrix[['Outcome', 'Average Price Move', 'probability', 'expected_value']]
matrix

,Outcome,Average Price Move,probability,expected_value
0,Loss,13.500000,0.375,5.062500
1,Win,20.957983,0.625,13.098739


In [232]:
data.groupby(['agreement'])['price_change'].sum()

agreement
0     972.0
1    2494.0
Name: price_change, dtype: float64

In [231]:
data

,date,hour,Open,Close,k,d,price_change,stochastic_change,price_direction,stochastic_direction,agreement
1,2022-04-03,12,NaN,4530.75,NaN,NaN,NaN,NaN,0,0,1
24,2022-04-04,12,4570.50,4573.25,97.260657,92.569858,2.75,4.690799,1,1,1
47,2022-04-05,12,4534.50,4523.00,9.923251,9.722959,11.50,0.200293,0,1,0
70,2022-04-06,12,4489.75,4461.75,37.337769,27.579343,28.00,9.758426,0,1,0
93,2022-04-07,12,4501.75,4501.50,81.536621,58.728534,0.25,22.808087,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
4266,2022-12-22,12,3826.00,3839.50,27.035623,14.816924,13.50,12.218700,1,1,1
4289,2022-12-26,12,3867.25,3890.75,86.254904,79.663741,23.50,6.591163,1,1,1
4312,2022-12-27,12,3854.50,3852.25,26.745718,30.830040,2.25,-4.084321,0,0,1
4335,2022-12-28,12,3824.75,3810.50,14.947215,19.585051,14.25,-4.637836,0,0,1


### Testing

In [ ]:
tmp = pd.DataFrame({'col1': [1, 2, 3, 4, 5], 'col2': [6, 7, 8, 9, 10]})
tmp['col1'] = tmp['col1'].shift(1)
tmp

In [94]:
data['price_difference'] = data['Close'] - data['Open']

In [95]:
data['price_direction'] = [1 if x > 0 else 0 for x in data['price_difference']]

In [96]:
data['stochastic_difference'] = data['k'] - data['d']

In [97]:
data['stochastic_direction'] = [1 if x > 0 else 0 for x in data['stochastic_difference']]

In [98]:
data

,Open,Close,k,d,price_difference,price_direction,stochastic_difference,stochastic_direction
Datetime,,,,,,,,
2022-01-02 18:00:00,4771.00,4775.00,NaN,NaN,4.00,1,NaN,0
2022-01-02 19:00:00,4775.00,4783.00,NaN,NaN,8.00,1,NaN,0
2022-01-02 20:00:00,4783.00,4778.00,NaN,NaN,-5.00,0,NaN,0
2022-01-02 21:00:00,4777.75,4779.50,NaN,NaN,1.75,1,NaN,0
2022-01-02 22:00:00,4779.50,4778.25,NaN,NaN,-1.25,0,NaN,0
...,...,...,...,...,...,...,...,...
2022-12-30 12:00:00,3845.25,3847.50,40.711743,34.883314,2.25,1,5.828429,1
2022-12-30 13:00:00,3847.50,3830.25,36.136018,37.647479,-17.25,0,-1.511460,0
2022-12-30 14:00:00,3830.50,3828.00,27.427351,34.758371,-2.50,0,-7.331020,0


In [99]:
data = data.reset_index()
data['Datetime'] = data['Datetime'] + timedelta(hours = -7)#.to_clipboard(index=False)
data

,Datetime,Open,Close,k,d,price_difference,price_direction,stochastic_difference,stochastic_direction
0,2022-01-02 11:00:00,4771.00,4775.00,NaN,NaN,4.00,1,NaN,0
1,2022-01-02 12:00:00,4775.00,4783.00,NaN,NaN,8.00,1,NaN,0
2,2022-01-02 13:00:00,4783.00,4778.00,NaN,NaN,-5.00,0,NaN,0
3,2022-01-02 14:00:00,4777.75,4779.50,NaN,NaN,1.75,1,NaN,0
4,2022-01-02 15:00:00,4779.50,4778.25,NaN,NaN,-1.25,0,NaN,0
...,...,...,...,...,...,...,...,...,...
5821,2022-12-30 05:00:00,3845.25,3847.50,40.711743,34.883314,2.25,1,5.828429,1
5822,2022-12-30 06:00:00,3847.50,3830.25,36.136018,37.647479,-17.25,0,-1.511460,0
5823,2022-12-30 07:00:00,3830.50,3828.00,27.427351,34.758371,-2.50,0,-7.331020,0
5824,2022-12-30 08:00:00,3828.00,3861.50,42.742990,35.435453,33.50,1,7.307537,1


In [102]:
data.to_clipboard(index=False)

In [118]:
data['hour'] = [x.hour for x in data['Datetime']]

In [121]:
data['Datetime'][0]#.day

Timestamp('2022-01-02 11:00:00')

In [124]:
datetime.strptime(data['Datetime'][0], '%Y-%m-%d')

TypeError: strptime() argument 1 must be str, not Timestamp

In [128]:
data['date'] = [x.date() for x in data['Datetime']]

In [129]:
data

,Datetime,Open,Close,k,d,price_difference,price_direction,stochastic_difference,stochastic_direction,hour,date
0,2022-01-02 11:00:00,4771.00,4775.00,NaN,NaN,4.00,1,NaN,0,11,2022-01-02
1,2022-01-02 12:00:00,4775.00,4783.00,NaN,NaN,8.00,1,NaN,0,12,2022-01-02
2,2022-01-02 13:00:00,4783.00,4778.00,NaN,NaN,-5.00,0,NaN,0,13,2022-01-02
3,2022-01-02 14:00:00,4777.75,4779.50,NaN,NaN,1.75,1,NaN,0,14,2022-01-02
4,2022-01-02 15:00:00,4779.50,4778.25,NaN,NaN,-1.25,0,NaN,0,15,2022-01-02
...,...,...,...,...,...,...,...,...,...,...,...
5821,2022-12-30 05:00:00,3845.25,3847.50,40.711743,34.883314,2.25,1,5.828429,1,5,2022-12-30
5822,2022-12-30 06:00:00,3847.50,3830.25,36.136018,37.647479,-17.25,0,-1.511460,0,6,2022-12-30
5823,2022-12-30 07:00:00,3830.50,3828.00,27.427351,34.758371,-2.50,0,-7.331020,0,7,2022-12-30
5824,2022-12-30 08:00:00,3828.00,3861.50,42.742990,35.435453,33.50,1,7.307537,1,8,2022-12-30


In [132]:
tmp = data[['date', 'hour', 'Open', 'Close', 'price_direction', 'stochastic_direction']]

In [135]:
tmp = filter_all(tmp, {'hour': [8, 12]}).sort_values('date')

In [136]:
tmp

,date,hour,Open,Close,price_direction,stochastic_direction
1,2022-01-02,12,4775.00,4783.00,1,0
21,2022-01-03,8,4781.25,4786.75,1,1
24,2022-01-03,12,4783.00,4784.25,1,0
44,2022-01-04,8,4782.50,4783.50,1,1
47,2022-01-04,12,4778.25,4776.75,0,0
...,...,...,...,...,...,...
5778,2022-12-28,8,3824.75,3807.00,0,0
5781,2022-12-28,12,3812.25,3810.50,0,1
5804,2022-12-29,12,3866.00,3866.25,1,0
5801,2022-12-29,8,3874.25,3872.00,0,0


In [141]:
tmp['price'] = [y if x == 8 else z for x, y, z in zip(tmp['hour'], tmp['Open'], tmp['Close'])]

In [142]:
tmp

,date,hour,Open,Close,price_direction,stochastic_direction,price
1,2022-01-02,12,4775.00,4783.00,1,0,4783.00
21,2022-01-03,8,4781.25,4786.75,1,1,4781.25
24,2022-01-03,12,4783.00,4784.25,1,0,4784.25
44,2022-01-04,8,4782.50,4783.50,1,1,4782.50
47,2022-01-04,12,4778.25,4776.75,0,0,4776.75
...,...,...,...,...,...,...,...
5778,2022-12-28,8,3824.75,3807.00,0,0,3824.75
5781,2022-12-28,12,3812.25,3810.50,0,1,3810.50
5804,2022-12-29,12,3866.00,3866.25,1,0,3866.25
5801,2022-12-29,8,3874.25,3872.00,0,0,3874.25


In [144]:
tmp['price_difference'] = tmp['price'].diff()

In [146]:
tmp['price_direction'] = [1 if x > 0 else 0 for x in tmp['price_difference']]

In [147]:
tmp

,date,hour,Open,Close,price_direction,stochastic_direction,price,price_difference
1,2022-01-02,12,4775.00,4783.00,0,0,4783.00,NaN
21,2022-01-03,8,4781.25,4786.75,0,1,4781.25,-1.75
24,2022-01-03,12,4783.00,4784.25,1,0,4784.25,3.00
44,2022-01-04,8,4782.50,4783.50,0,1,4782.50,-1.75
47,2022-01-04,12,4778.25,4776.75,0,0,4776.75,-5.75
...,...,...,...,...,...,...,...,...
5778,2022-12-28,8,3824.75,3807.00,0,0,3824.75,-27.50
5781,2022-12-28,12,3812.25,3810.50,0,1,3810.50,-14.25
5804,2022-12-29,12,3866.00,3866.25,1,0,3866.25,55.75
5801,2022-12-29,8,3874.25,3872.00,1,0,3874.25,8.00


In [151]:
tmp['new_stochastic_direction'] = tmp['stochastic_direction'].shift(-1)

In [153]:
tmp.sort_values('date')

,date,hour,Open,Close,price_direction,stochastic_direction,price,price_difference,new_stochastic_direction
1,2022-01-02,12,4775.00,4783.00,0,0,4783.00,NaN,1.0
21,2022-01-03,8,4781.25,4786.75,0,1,4781.25,-1.75,0.0
24,2022-01-03,12,4783.00,4784.25,1,0,4784.25,3.00,1.0
44,2022-01-04,8,4782.50,4783.50,0,1,4782.50,-1.75,0.0
47,2022-01-04,12,4778.25,4776.75,0,0,4776.75,-5.75,0.0
...,...,...,...,...,...,...,...,...,...
5778,2022-12-28,8,3824.75,3807.00,0,0,3824.75,-27.50,1.0
5781,2022-12-28,12,3812.25,3810.50,0,1,3810.50,-14.25,0.0
5801,2022-12-29,8,3874.25,3872.00,1,0,3874.25,8.00,1.0
5804,2022-12-29,12,3866.00,3866.25,1,0,3866.25,55.75,0.0


In [173]:
new_tmp = tmp[['date', 'hour', 'Open', 'Close', 'price_direction', 'new_stochastic_direction']]

In [174]:
analysis_data = filter_all(new_tmp, {'hour': [12]})

In [175]:
analysis_data['pip_move'] = analysis_data['Close'] - analysis_data['Open']

C:\Users\Regan-John Daniels\AppData\Local\Temp\ipykernel_13372\3885480508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_data['pip_move'] = analysis_data['Close'] - analysis_data['Open']


In [190]:
analysis_data['agreement'] = [1 if x == y else 0 for x, y in zip(analysis_data['price_direction'], analysis_data['new_stochastic_direction'])]

C:\Users\Regan-John Daniels\AppData\Local\Temp\ipykernel_13372\3802003544.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_data['agreement'] = [1 if x == y else 0 for x, y in zip(analysis_data['price_direction'], analysis_data['new_stochastic_direction'])]


In [186]:
analysis_data.groupby(['agreement'])['pip_move'].mean()

agreement
0    4.982456
1    4.909574
Name: pip_move, dtype: float64

In [179]:
analysis_data.to_clipboard(index=False)

In [172]:
tmp.to_clipboard(index=False)

In [185]:
analysis_data['pip_move'] = analysis_data['pip_move'].abs()

C:\Users\Regan-John Daniels\AppData\Local\Temp\ipykernel_13372\1390237951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_data['pip_move'] = analysis_data['pip_move'].abs()


In [188]:
analysis_data['price_change'] = analysis_data['Close'] - analysis_data['Open']
analysis_data['price_direction'] = [1 if x > 0 else 0 for x in analysis_data['price_change']]

C:\Users\Regan-John Daniels\AppData\Local\Temp\ipykernel_13372\725473531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_data['price_change'] = analysis_data['Close'] - analysis_data['Open']
C:\Users\Regan-John Daniels\AppData\Local\Temp\ipykernel_13372\725473531.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  analysis_data['price_direction'] = [1 if x > 0 else 0 for x in analysis_data['price_change']]


In [191]:
analysis_data

,date,hour,Open,Close,price_direction,new_stochastic_direction,pip_move,agreement,price_change
1,2022-01-02,12,4775.00,4783.00,1,1.0,8.00,1,8.00
24,2022-01-03,12,4783.00,4784.25,1,1.0,1.25,1,1.25
47,2022-01-04,12,4778.25,4776.75,0,0.0,1.50,1,-1.50
70,2022-01-05,12,4698.00,4698.25,1,1.0,0.25,1,0.25
93,2022-01-06,12,4699.25,4703.75,1,0.0,4.50,0,4.50
...,...,...,...,...,...,...,...,...,...
5712,2022-12-22,12,3848.50,3839.50,0,1.0,9.00,0,-9.00
5735,2022-12-26,12,3893.75,3890.75,0,0.0,3.00,1,-3.00
5758,2022-12-27,12,3855.50,3852.25,0,0.0,3.25,1,-3.25
5781,2022-12-28,12,3812.25,3810.50,0,0.0,1.75,1,-1.75


In [192]:
analysis_data.groupby(['agreement'])['agreement'].count()

agreement
0    128
1    127
Name: agreement, dtype: int64